In this notebook, I load both the bicycles dataset and the pollution dataset and take a quick look at them.

I discover that the pollution dataset contains pollution readings from about 25 weather stations across Seoul, and I have to decide whether to create an average pollution score or select a single weather station. After plotting the weather station locations on a map, I decide to select a single central weather station, reasoning that most bicycle hire stations will in the central downtown area of Seoul.

Finally, I process the time variables in both datasets then append the pollution data to the bicycles dataset.

Dataset saved here: 'data/interim/01-merged-data.csv'

In [1]:
import pandas as pd

In [2]:
# Use if working in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Use if working in colab
# set working directory to project directory
%cd '/content/drive/My Drive/github/bicycles-seoul'

/content/drive/My Drive/github/bicycles-seoul


In [4]:
df_bicycles = pd.read_csv('data/raw/bicycles_seoul_raw.csv')
df_bicycles.head()

,Hour,Temperature(C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,rented_bike_count,day,month,year,week_day,timestamp
0,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,254,1,12,2017,4,1512086400
1,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,204,1,12,2017,4,1512086400
2,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,173,1,12,2017,4,1512086400
3,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,107,1,12,2017,4,1512086400
4,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,78,1,12,2017,4,1512086400


In [5]:
df_pollution = pd.read_csv('data/external/seoul-pollution.csv')
df_pollution.head()

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5
0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0
1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0
2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0
3,2017-01-01 03:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,58.0
4,2017-01-01 04:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.003,0.051,0.002,1.2,69.0,61.0


In [6]:
# How many weather stations do we have here?

unique_addresses = df_pollution['Address'].unique()

# Display the unique addresses
print("Unique Addresses in the Pollution Dataset:")
for address in unique_addresses:
    print(address)

Unique Addresses in the Pollution Dataset:
19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea
15, Deoksugung-gil, Jung-gu, Seoul, Republic of Korea
136, Hannam-daero, Yongsan-gu, Seoul, Republic of Korea
215, Jinheung-ro, Eunpyeong-gu, Seoul, Republic of Korea
32, Segeomjeong-ro 4-gil, Seodaemun-gu, Seoul, Republic of Korea
10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic of Korea
18, Ttukseom-ro 3-gil, Seongdong-gu, Seoul, Republic of Korea
571, Gwangnaru-ro, Gwangjin-gu, Seoul, Republic of Korea
43, Cheonho-daero 13-gil, Dongdaemun-gu, Seoul, Republic of Korea
369, Yongmasan-ro, Jungnang-gu, Seoul, Republic of Korea
70, Samyang-ro 2-gil, Seongbuk-gu, Seoul, Republic of Korea
49, Samyang-ro 139-gil, Gangbuk-gu, Seoul, Republic of Korea
34, Sirubong-ro 2-gil, Dobong-gu, Seoul, Republic of Korea
17, Sanggye-ro 23-gil, Nowon-gu, Seoul, Republic of Korea
56, Jungang-ro 52-gil, Yangcheon-gu, Seoul, Republic of Korea
71, Gangseo-ro 45da-gil, Gangseo-gu, Seoul, Republic of Korea
45, Gamas

In [7]:
# So, looking at the readout below it seems the dataset includes hourly pollution readings from every station.
# I'm going to need to either average pollution readings from all stations, or select a single station for my analysis.

# Convert Measurement date to datetime
df_pollution['Measurement date'] = pd.to_datetime(df_pollution['Measurement date'])

# Group by 'Measurement date' and count unique addresses
timestamp_address_counts = df_pollution.groupby('Measurement date')['Address'].nunique()

# Check which timestamps have more than one unique address
multi_address_timestamps = timestamp_address_counts[timestamp_address_counts > 1]

if len(multi_address_timestamps) > 0:
    print("These timestamps have multiple readings from different addresses:")
    print(multi_address_timestamps)
else:
    print("No timestamps found with multiple readings from different addresses.")

These timestamps have multiple readings from different addresses:
Measurement date
2017-01-01 00:00:00    25
2017-01-01 01:00:00    25
2017-01-01 02:00:00    25
2017-01-01 03:00:00    25
2017-01-01 04:00:00    25
                       ..
2019-12-31 19:00:00    25
2019-12-31 20:00:00    25
2019-12-31 21:00:00    25
2019-12-31 22:00:00    25
2019-12-31 23:00:00    25
Name: Address, Length: 25906, dtype: int64


In [8]:
# Number of hours between the start and end dates. Pretty similar.
# I'm going to assume that all stations have readings for all timestamps, then.

start = pd.to_datetime("2017-01-01 00:00:00")
end = pd.to_datetime("2019-12-31 23:00:00")

diff = end - start
hours = diff.total_seconds() / 3600
print(hours)

26279.0


In [9]:
# Let's create a quick folium map of all the weather stations to see if we could select one central station for this project.
# Looking at the map we've produced, the station '136, Hannam-daero, Yongsan-gu, Seoul, Republic of Korea' looks ideally central.

import folium
from geopy.geocoders import Nominatim

addresses = [
    "19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republic of Korea",
    "15, Deoksugung-gil, Jung-gu, Seoul, Republic of Korea",
    "136, Hannam-daero, Yongsan-gu, Seoul, Republic of Korea",
    "215, Jinheung-ro, Eunpyeong-gu, Seoul, Republic of Korea",
    "32, Segeomjeong-ro 4-gil, Seodaemun-gu, Seoul, Republic of Korea",
    "10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic of Korea",
    "18, Ttukseom-ro 3-gil, Seongdong-gu, Seoul, Republic of Korea",
    "571, Gwangnaru-ro, Gwangjin-gu, Seoul, Republic of Korea",
    "43, Cheonho-daero 13-gil, Dongdaemun-gu, Seoul, Republic of Korea",
    "369, Yongmasan-ro, Jungnang-gu, Seoul, Republic of Korea",
    "70, Samyang-ro 2-gil, Seongbuk-gu, Seoul, Republic of Korea",
    "49, Samyang-ro 139-gil, Gangbuk-gu, Seoul, Republic of Korea",
    "34, Sirubong-ro 2-gil, Dobong-gu, Seoul, Republic of Korea",
    "17, Sanggye-ro 23-gil, Nowon-gu, Seoul, Republic of Korea",
    "56, Jungang-ro 52-gil, Yangcheon-gu, Seoul, Republic of Korea",
    "71, Gangseo-ro 45da-gil, Gangseo-gu, Seoul, Republic of Korea",
    "45, Gamasan-ro 27-gil, Guro-gu, Seoul, Republic of Korea",
    "20, Geumha-ro 21-gil, Geumcheon-gu, Seoul, Republic of Korea",
    "11, Yangsan-ro 23-gil, Yeongdeungpo-gu, Seoul, Republic of Korea",
    "6, Sadang-ro 16a-gil, Dongjak-gu, Seoul, Republic of Korea",
    "14, Sillimdong-gil, Gwanak-gu, Seoul, Republic of Korea",
    "16, Sinbanpo-ro 15-gil, Seocho-gu, Seoul, Republic of Korea",
    "26, Hakdong-ro, Gangnam-gu, Seoul, Republic of Korea",
    "236, Baekjegobun-ro, Songpa-gu, Seoul, Republic of Korea",
    "59, Gucheonmyeon-ro 42-gil, Gangdong-gu, Seoul, Republic of Korea"
]

# Initialize geocoder
geolocator = Nominatim(user_agent="address_mapper")

# Geocode each address
locations = []
for addr in addresses:
    location = geolocator.geocode(addr)
    if location:
        locations.append((location.latitude, location.longitude))
    else:
        print(f"Could not geocode: {addr}")

# Create a Folium map centered on Seoul
# (Coordinates roughly for central Seoul)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# Add markers for each location
for i, (lat, lon) in enumerate(locations):
    folium.Marker([lat, lon], popup=addresses[i]).add_to(m)

# Save the map to an HTML file
m.save("seoul_addresses_map.html")

print("Map created: seoul_addresses_map.html")

Could not geocode: 34, Sirubong-ro 2-gil, Dobong-gu, Seoul, Republic of Korea
Map created: seoul_addresses_map.html


In [10]:
# Drop all addresses other than the central address we've selected.

df_pollution = df_pollution[df_pollution['Address'] == "136, Hannam-daero, Yongsan-gu, Seoul, Republic of Korea"]

In [11]:
# Formatting for pollution data, to ensure compatibility. Then use floor to bring it down to hours.
df_pollution['Measurement date'] = pd.to_datetime(df_pollution['Measurement date'], format='%Y-%m-%d %H:%M')
df_pollution['Measurement date'] = df_pollution['Measurement date'].dt.floor('h')

# Ensuring same formatting for bicycles.
df_bicycles['datetime'] = pd.to_datetime(df_bicycles[['year', 'month', 'day', 'Hour']])
df_bicycles['datetime'] = df_bicycles['datetime'].dt.floor('h')

In [12]:
# Merge on the date and time
merged_df = pd.merge(
    df_bicycles,
    df_pollution[['Measurement date', 'SO2', 'NO2', 'O3', 'CO', 'PM10', 'PM2.5']],
    left_on='datetime',
    right_on='Measurement date',
    how='left'
)

In [13]:
# Check merge makes sense.
merged_df.head()

,Hour,Temperature(C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,...,week_day,timestamp,datetime,Measurement date,SO2,NO2,O3,CO,PM10,PM2.5
0,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,...,4,1512086400,2017-12-01 00:00:00,2017-12-01 00:00:00,0.003,0.017,0.022,0.3,17.0,9.0
1,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,...,4,1512086400,2017-12-01 01:00:00,2017-12-01 01:00:00,0.003,0.018,0.020,0.3,19.0,11.0
2,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,...,4,1512086400,2017-12-01 02:00:00,2017-12-01 02:00:00,0.003,0.016,0.020,0.3,15.0,13.0
3,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,...,4,1512086400,2017-12-01 03:00:00,2017-12-01 03:00:00,0.003,0.015,0.021,0.3,19.0,13.0
4,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,...,4,1512086400,2017-12-01 04:00:00,2017-12-01 04:00:00,0.004,0.016,0.022,0.3,13.0,7.0


In [14]:
# Drop the redundant 'Measurement date' column
merged_df.drop(columns=['Measurement date'], inplace=True)

In [20]:
#Quick check that the number of cases of original dataset and merged dataset is the same...
print(df_bicycles.shape)
print(merged_df.shape)

(8760, 19)
(8760, 25)


In [15]:
# Save as csv in Drive
merged_df.to_csv('data/interim/01-merged-data.csv', index=False)